# **EDA analysis of Windows Log**

In [1]:
# EDA analysis of Windows Log

In [1]:
#imprting libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
# configuring matplotlib for inline plotting means that the plots will be displayed in the notebook
%matplotlib inline

In [7]:
# configuring 
# defining base directory as current working directory
base_dir = os.getcwd()
# defining the path to the log file data .csv file
log_file_path = os.path.join(base_dir,'data','WindowsEventLogs_Last30Days.csv')


In [8]:
# reading the log file data into a pandas dataframe
log_df = pd.read_csv(log_file_path)
print(log_df.head())

           TimeCreated     Id LevelDisplayName  \
0  21-05-2025 10:44:41  16384      Information   
1  21-05-2025 10:44:11  16394      Information   
2  21-05-2025 10:40:08      1      Information   
3  21-05-2025 10:40:08      1      Information   
4  21-05-2025 10:40:07      1      Information   

                     ProviderName  \
0  Microsoft-Windows-Security-SPP   
1  Microsoft-Windows-Security-SPP   
2                 Universal Print   
3                 Universal Print   
4                 Universal Print   

                                             Message      LogName  
0  Successfully scheduled Software Protection ser...  Application  
1             Offline downlevel migration succeeded.  Application  
2  Initialization Successful. Enabled=false, Clou...  Application  
3  Universal Print is not enabled in the user's t...  Application  
4  Successfully received access_token for 0000000...  Application  


we have dataframe with 6 columns and 49201 rows 

In [10]:
# checking the shape of the dataframe
print(log_df.shape)

(49201, 6)


we have columns TimeCreated, Id, LevelDisplayName, ProviderName, Message, LogName with data type as object except Id which is int64

In [11]:
# columns of dataframe
print(log_df.columns)

Index(['TimeCreated', 'Id', 'LevelDisplayName', 'ProviderName', 'Message',
       'LogName'],
      dtype='object')


In [12]:
# checking the data types of the columns
print(log_df.dtypes)

TimeCreated         object
Id                   int64
LevelDisplayName    object
ProviderName        object
Message             object
LogName             object
dtype: object


In [15]:
# we will do data cleaning and data preprocessing

# checking for null values in the dataframe
# we will also print the row of the dataframe which has null values
print(log_df[log_df.isnull().any(axis=1)])
print(log_df.isnull().sum())

               TimeCreated    Id LevelDisplayName              ProviderName  \
63     21-05-2025 10:30:40  9997      Information               Application   
95     21-05-2025 10:29:11     0              NaN  RtkAudioUniversalService   
96     21-05-2025 10:29:11     0              NaN           SynTPEnhService   
108    21-05-2025 03:19:51     0              NaN  RtkAudioUniversalService   
115    21-05-2025 03:19:49     0              NaN           SynTPEnhService   
...                    ...   ...              ...                       ...   
49127  22-04-2025 02:30:55  7021      Information                  Netwtw14   
49159  21-04-2025 22:43:44  7021      Information                  Netwtw14   
49161  21-04-2025 22:41:14  7021      Information                  Netwtw14   
49166  21-04-2025 22:37:56  7021      Information                  Netwtw14   
49195  21-04-2025 19:43:04  7021      Information                  Netwtw14   

      Message      LogName  
63        NaN  Applica

after removing the null values we will have 48231 rows and 6 columns 

In [19]:
# we will drop the null values from the dataframe
log_df = log_df.dropna()
# checking the shape of the dataframe after dropping null values
print(log_df.shape)
print(log_df.isnull().sum())


(48231, 6)
TimeCreated         0
Id                  0
LevelDisplayName    0
ProviderName        0
Message             0
LogName             0
dtype: int64


# **now we will do Analysis of the data with different columns and their values in diffent ways**

first we will do analysis of the data with respect to TimeCreated column
# *Timestamp Analysis*

In [23]:
# converting the TimeCreated column to datetime format
log_df= log_df.copy()
log_df['TimeCreated'] = pd.to_datetime(
    log_df['TimeCreated'],
    format='%d-%m-%Y %H:%M:%S'
) #year/month/day hour/minute/
print(log_df['TimeCreated'].head())
print(log_df.dtypes)

0   2025-05-21 10:44:41
1   2025-05-21 10:44:11
2   2025-05-21 10:40:08
3   2025-05-21 10:40:08
4   2025-05-21 10:40:07
Name: TimeCreated, dtype: datetime64[ns]
TimeCreated         datetime64[ns]
Id                           int64
LevelDisplayName            object
ProviderName                object
Message                     object
LogName                     object
dtype: object


In [24]:
# TimeCreated column is converted to datetime format datetime64[ns]


In [26]:
# minimum and maximum date in the TimeCreated column
min_date= log_df['TimeCreated'].min()
max_date= log_df['TimeCreated'].max()

2025-04-21 19:43:02
2025-05-21 10:51:28


## *Temporal Event Sequence Analysis*
goal: Identify patterns in the order/timing of events

### *method*
1. Lag analysis:Compute time differences between consecutive events of specific Id or ProviderName (e.g., security-related events like 4624 [logon] and 4625 [failed logon]).
2. Markov Chains: Model transition probabilities between event types (e.g., likelihood of an "Error" following a "Warning").
3. Sequence Mining: Use algorithms like SPADE to discover frequent event sequences (e.g., repeated failed logons followed by a successful logon).

In [ ]:
#we will sort the dataframe by TimeCreated column
log_df =log_df.sorted_values(by='TimeCreated')

#lag analysis
